In [1]:
import json

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

endpoint = "https://ocr-ee365.cognitiveservices.azure.com/"
key = "69560f077d394e1ead3a5b0c48926c72"

model_id = "CHEQUE-OCR-4"
formUrl = "https://eagleeye365ocrstorage.blob.core.windows.net/sourcefiles/cheque5.png"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Make sure your document's type is included in the list of document types the custom model can analyze
poller = document_analysis_client.begin_analyze_document_from_url(model_id, formUrl)
result = poller.result()

In [3]:
print(result)

AnalyzeResult(api_version=2022-08-31, model_id=CHEQUE-OCR-4, content=FLEET FIRST !!
DEMO COMPANY INC 1 MAIN ST A CITY, A STATE, 12321
PAY TO THE ORDER OF
CUMBERLAND COUNTY GENERAL FUND
INSERT Bank of 5th Street
FOR
฿: 012345678910
012345678910
881724
06/06/2018
$ 67,979.00
DOLLARS
Jane Doer
881724, languages=[], pages=[DocumentPage(page_number=1, angle=None, width=858.0, height=374.0, unit=pixel, lines=[DocumentLine(content=FLEET, polygon=[Point(x=43.0, y=36.0), Point(x=100.0, y=35.0), Point(x=99.0, y=57.0), Point(x=42.0, y=57.0)], spans=[DocumentSpan(offset=0, length=5)]), DocumentLine(content=FIRST !!, polygon=[Point(x=42.0, y=66.0), Point(x=113.0, y=65.0), Point(x=113.0, y=85.0), Point(x=42.0, y=86.0)], spans=[DocumentSpan(offset=6, length=8)]), DocumentLine(content=DEMO COMPANY INC, polygon=[Point(x=130.0, y=42.0), Point(x=297.0, y=42.0), Point(x=297.0, y=59.0), Point(x=130.0, y=59.0)], spans=[DocumentSpan(offset=15, length=16)]), DocumentLine(content=1 MAIN ST, polygon=[Point(x=13

In [11]:
full_json=[]

In [12]:
for idx, document in enumerate(result.documents):
    json_array=[]
    json_object={}
    prev_json_object={}
    main_json={}
    main_json['document_num'] = idx + 1
    main_json['confidence'] = document.confidence
    main_json['model_name'] = result.model_id
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        if(field.value_type != 'list'):
            main_json[name] = field_value
#             print("found field of name '{}' with value '{}'\n".format(name, field_value))
        else:
            for x in field_value:
                json_object={}
                for z in x.value:
                    try:
                        json_object[z] = x.value[z].value if x.value[z].value != None else prev_json_object[z]
                    except:
                        json_object[z] = x.value[z].value
                prev_json_object = json_object
                json_array.append(json_object)
        main_json['entries'] = json_array
full_json.append(main_json)


In [13]:
print(full_json)

[{'document_num': 1, 'confidence': 0.992, 'model_name': 'CHEQUE-OCR-4', 'BANK': 'Bank of 5th Street', 'entries': [], 'PAY_TO': 'CUMBERLAND COUNTY GENERAL FUND', 'CURRENCY': 'DOLLARS', 'AMOUNT': '67,979.00', 'CHEQUE_NO': '881724', 'SIGNATURE': 'Jane Doer', 'ADDRESS': 'DEMO COMPANY INC 1 MAIN ST A CITY, A STATE, 12321', 'DATE': '06/06/2018'}]
